In [2]:
pip install pandas openpyxl sqlalchemy psycopg2

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

In [4]:
# Specify base directory
base_directory = r'C:\Users\julia\Projects\SRoses\Production'

def read_files(base_directory, production_files=False):
    data_frames = []
    for root, dirs, files in os.walk(base_directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                if production_files:
                    # Only include files named "Production"
                    if file.lower().startswith('production') and file.endswith('.csv'):
                        df = pd.read_csv(file_path)
                        data_frames.append(df)
                else:
                    # Exclude files named "Production"
                    if file.lower().startswith('production'):
                        continue
                    if file.endswith('.xlsx') or file.endswith('.xls'):
                        df = pd.read_excel(file_path)
                    elif file.endswith('.csv'):
                        df = pd.read_csv(file_path)
                    else:
                        continue
                    
                    # Extract year, month, day from date column
                    if 'Date' in df.columns:
                        df['Year'] = pd.to_datetime(df['Date']).dt.year
                        df['Month'] = pd.to_datetime(df['Date']).dt.month
                        df['Day'] = pd.to_datetime(df['Date']).dt.day
                        
                        # Filter data for years 2022, 2023, and 2024
                        df = df[df['Year'].isin([2022, 2023, 2024])]
                        
                    data_frames.append(df)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
    return data_frames

In [5]:
def save_and_upload(df, file_name, table_name, engine):
    os.makedirs('data/processed', exist_ok=True)
    csv_path = f'data/processed/{file_name}'
    df.to_csv(csv_path, index=False)
    print(f"Combined data saved successfully to {csv_path}.")
    
    try:
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)
        print(f"Data loaded into SQL Server successfully into table {table_name}.")
    except Exception as e:
        print(f"Error loading data into SQL Server: {e}")

In [6]:
if __name__ == "__main__":
    # Define SQL Server connection parameters
    server = 'server' #include needed server
    database = 'database' #include needed database
    username = 'username' #include needed username
    password = 'password' #include needed password
    driver = 'ODBC Driver 17 for SQL Server'

    # Create SQLAlchemy engine
    connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}'
    engine = create_engine(connection_string)

    # Read non-Production files
    data_frames = read_files(base_directory)
    if data_frames:
        combined_df = pd.concat(data_frames, ignore_index=True)
        save_and_upload(combined_df, 'combined_data.csv', 'combined_data', engine)
    else:
        print("No data frames to combine.")

Combined data saved successfully to data/processed/combined_data.csv.
Data loaded into SQL Server successfully into table combined_data.
